In [3]:
%%file xml2dict.py

#Written by Baptiste Lemaire
import xml.etree.ElementTree as ET

class xml2dict:
    """
    xml2dict turns an XML file into a array of all the species involved into the system of
    reactions and a list of dictionaries. This latter contains a dictionary for every single 
    reaction of the system of reactions. Each dictionary contains the information about the 
    reactants, the products, the stoichiometric coefficients and the kinetic law for the 
    reaction constant (Arrhenius Law, Modified Arrhenius Law, Constant Law, and so forth).
    where list of dictionary Reaction contains all the infomation from one given reaction.
    
    INPUTS
    ======
    
    file: XML file.
          The format of the this input file must follow
          the format given by Prof. David Sondak.
          
    METHODS
    =======
    
    parse(self, file): 
        extract all the information contained into the XML file provided by
        the user, namely the names of all the species involved in the system
        of reactions, stored into the array self.Species, and all the information
        about every reaction, stored into self.ListDictionaries
        
    get_info(self):
        return the array self.Species containing the names of the species involved
        and the list of dictionaries self.ListDictionaries
        OUTPUTS: self.Species, self.ListDictionaries
           
    __repr__(self):
        return a string of all the sppecies and all the information contained in 
        self.dictionaries
        OUTPUTS: str
    
    
    EXAMPLES
    =========
    >>> reader = xml2dict()
    >>> reader.parse('rxns.xml')
    >>> reader.get_info()
    (['H', 'O', 'OH', 'H2', 'O2'], [{'coeffParams': {'A': 35200000000.0, 'b': -0.7, 'E': 71400.0}, 'coeffUnits': {'A': 'm3/mol/s', 'b': 'dimensionless', 'E': 'J/mol'}, 'ID': 'reaction01', 'reversible': 'yes', 'TYPE': 'Elementary', 'reactants': {'H': 1, 'O2': 1}, 'products': {'OH': 1, 'O': 1}, 'coeffLaw': 'Arrhenius'}, {'coeffParams': {'A': 0.0506, 'b': 2.7, 'E': 26300.0}, 'coeffUnits': {'A': 'm3/mol/s', 'b': 'dimensionless', 'E': 'J/mol'}, 'ID': 'reaction02', 'reversible': 'yes', 'TYPE': 'Elementary', 'reactants': {'H2': 1, 'O': 1}, 'products': {'OH': 1, 'H': 1}, 'coeffLaw': 'Arrhenius'}])
    """

    def parse(self, file):
        self.file = file
        tree = ET.parse(file)
        root = tree.getroot()

        SpeciesArray=root.find('phase').find('speciesArray')
        self.Species = SpeciesArray.text.strip().split(" ")
    
        self.ListDictionaries = []
    
        #Now: go through every reaction to read the features:
    
        for reaction in root.find('reactionData').findall('reaction'):
        
            #Initialization of the variables
        
            Dict = {}
            reactants = []
            products = []
            Nup = []
            Nupp = []
            ListCoeffUnits = []
        
            ListReactants = reaction.find('reactants').text.split()
            for elementsR in ListReactants:
                specie, nu = elementsR.split(':')
                reactants.append(specie)
                Nup.append(int(nu))
            ListProducts = reaction.find('products').text.split()
            for elementsP in ListProducts:
                specie, nu = elementsP.split(':')
                products.append(specie)
                Nupp.append(int(nu))
            for name in reaction.find('rateCoeff'):
                Law = name.tag
                ListCoeffTag = []
                ListCoeffValue = []
                for coeff in name:
                    ListCoeffTag.append(coeff.tag)
                    ListCoeffValue.append(float(coeff.text))
                    if len(coeff.attrib) != 0:
                        ListCoeffUnits.append(coeff.attrib['units'])
                    else:
                        ListCoeffUnits.append('dimensionless')
            Dict['coeffParams'] = dict(zip(ListCoeffTag, ListCoeffValue))
            Dict['coeffUnits'] = dict(zip(ListCoeffTag,ListCoeffUnits))
            Dict['ID'] = reaction.attrib['id']
            Dict['reversible'] = reaction.attrib['reversible']
            Dict['TYPE'] = reaction.attrib['type']
            Dict['reactants'] = dict(zip(reactants, Nup))
            Dict['products'] = dict(zip(products, Nupp))
            Dict['coeffLaw'] = Law
            self.ListDictionaries.append(Dict)
    
    def get_info(self):
        return self.Species, self.ListDictionaries
    
    def __repr__(self):
        return str(self.Species) + ' ' + str(self.ListDictionaries)



Overwriting xml2dict.py


In [20]:
from xml2dict import xml2dict

reader = xml2dict()

In [29]:
!pytest --cov --cov-report term-missing

============================= test session starts ==============================
platform darwin -- Python 3.6.1, pytest-3.0.7, py-1.4.33, pluggy-0.4.0
rootdir: /Users/baptistelemaire/Documents/cs207-FinalProject, inifile: setup.cfg
plugins: cov-2.5.1
collected 26 items / 4 errors 


---------- coverage: platform darwin, python 3.6.1-final-0 -----------
Name                                       Stmts   Miss  Cover   Missing
------------------------------------------------------------------------
Reaction/BaseClass.py                         33     21    36%   74, 78, 82, 86, 90, 94, 98, 102-105, 109-114, 118-121
Reaction/Reaction.py                          79     55    30%   206-210, 212-224, 226-238, 249, 255, 258, 261-262, 277-288, 292, 294-300, 303, 305, 308, 311-322, 324-329, 332, 335-342
Reaction/_LawFunction.py                      48     28    42%   5, 7, 9, 13, 15, 17, 20-21, 28-30, 32-34, 36, 38, 42-46, 48-51, 53-54, 56
Reaction/test_Reaction.py                     88     76

In [22]:
reader.parse('rxns2.xml')

In [23]:
reader.get_info()

(['H2', 'O2', 'OH', 'HO2', 'H2O'],
 [{'ID': 'reaction01',
   'TYPE': 'Elementary',
   'coeffLaw': 'modArrhenius',
   'coeffParams': {'A': 100000000.0, 'E': 50000.0, 'b': 0.5},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol', 'b': 'dimensionless'},
   'products': {'H2': 1, 'OH': 2},
   'reactants': {'H2': 2, 'O2': 1},
   'reversible': 'yes'},
  {'ID': 'reaction02',
   'TYPE': 'Elementary',
   'coeffLaw': 'Constant',
   'coeffParams': {'k': 10000.0},
   'coeffUnits': {'k': 'm3/mol/s'},
   'products': {'H2O': 1, 'O2': 1},
   'reactants': {'HO2': 1, 'OH': 1},
   'reversible': 'yes'},
  {'ID': 'reaction03',
   'TYPE': 'Elementary',
   'coeffLaw': 'Arrhenius',
   'coeffParams': {'A': 10000000.0, 'E': 10000.0},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol'},
   'products': {'O': 1, 'OH': 1},
   'reactants': {'H': 1, 'O2': 1},
   'reversible': 'yes'}])

In [28]:
reader.get_info()[1][2]['reversible']

'yes'

In [91]:
repr(reader)

"['H2', 'O2', 'OH', 'HO2', 'H2O'] [{'coeffParams': {'A': 100000000.0, 'b': 0.5, 'E': 50000.0}, 'coeffUnits': {'A': 'm3/mol/s', 'b': 'dimensionless', 'E': 'J/mol'}, 'id': 'reaction01', 'reversible': 'yes', 'type': 'Elementary', 'reactants': {'H2': 2, 'O2': 1}, 'products': {'OH': 2, 'H2': 1}, 'coeffLaw': 'modArrhenius'}, {'coeffParams': {'k': 10000.0}, 'coeffUnits': {'k': 'm3/mol/s'}, 'id': 'reaction02', 'reversible': 'yes', 'type': 'Elementary', 'reactants': {'OH': 1, 'HO2': 1}, 'products': {'H2O': 1, 'O2': 1}, 'coeffLaw': 'Constant'}, {'coeffParams': {'A': 10000000.0, 'E': 10000.0}, 'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol'}, 'id': 'reaction03', 'reversible': 'yes', 'type': 'Elementary', 'reactants': {'H': 1, 'O2': 1}, 'products': {'OH': 1, 'O': 1}, 'coeffLaw': 'Arrhenius'}]"

In [85]:
dir(reader)

['ListDictionaries',
 'Species',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'file',
 'get_info',
 'parse']

In [7]:
r1 = xml2dict()
r1.parse('rxns.xml')
r1.get_info()

(['H', 'O', 'OH', 'H2', 'O2'],
 [{'ID': 'reaction01',
   'TYPE': 'Elementary',
   'coeffLaw': 'Arrhenius',
   'coeffParams': {'A': 35200000000.0, 'E': 71400.0, 'b': -0.7},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol', 'b': 'dimensionless'},
   'products': {'O': 1, 'OH': 1},
   'reactants': {'H': 1, 'O2': 1},
   'reversible': 'yes'},
  {'ID': 'reaction02',
   'TYPE': 'Elementary',
   'coeffLaw': 'Arrhenius',
   'coeffParams': {'A': 0.0506, 'E': 26300.0, 'b': 2.7},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol', 'b': 'dimensionless'},
   'products': {'H': 1, 'OH': 1},
   'reactants': {'H2': 1, 'O': 1},
   'reversible': 'yes'}])

In [17]:
r1.get_info()[1][0]['coeffLaw']

'Arrhenius'

In [9]:
r1.get_info() == (['H', 'O', 'OH', 'H2', 'O2'],
 [{'ID': 'reaction01',
   'TYPE': 'Elementary',
   'coeffLaw': 'Arrhenius',
   'coeffParams': {'A': 35200000000.0, 'E': 71400.0, 'b': -0.7},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol', 'b': 'dimensionless'},
   'products': {'O': 1, 'OH': 1},
   'reactants': {'H': 1, 'O2': 1},
   'reversible': 'yes'},
  {'ID': 'reaction02',
   'TYPE': 'Elementary',
   'coeffLaw': 'Arrhenius',
   'coeffParams': {'A': 0.0506, 'E': 26300.0, 'b': 2.7},
   'coeffUnits': {'A': 'm3/mol/s', 'E': 'J/mol', 'b': 'dimensionless'},
   'products': {'H': 1, 'OH': 1},
   'reactants': {'H2': 1, 'O': 1},
   'reversible': 'yes'}])

True